In [1]:
import pandas as pd
import stride
import datetime
import pytz
import math

date/time range - should be no more then a few hours to limit scale of the queries

In [2]:
datetime_from = pytz.timezone('israel').localize(datetime.datetime(2022, 4, 3, 10))
datetime_to = datetime_from + datetime.timedelta(hours=4)
print('date/time range')
print('from:', datetime_from)
print('to:', datetime_to)

date/time range
from: 2022-04-03 10:00:00+03:00
to: 2022-04-03 14:00:00+03:00


origin gridbox, this is roughly a 100m x 100m box

it's important to have a consistent logic to the grid box, so it can be used recursively

the logic here is each .001 lon/lat is a grid box, which is roughly 100m x 100m

In [3]:
origin_gridbox_lon_from = 34.808
origin_gridbox_lon_to = 34.809
origin_gridbox_lat_from = 31.961
origin_gridbox_lat_to = 31.962
print('origin grid box')
print('lon:', origin_gridbox_lon_from, '-', origin_gridbox_lon_to)
print('lat:', origin_gridbox_lat_from, '-', origin_gridbox_lat_to)

origin grid box
lon: 34.808 - 34.809
lat: 31.961 - 31.962


target gridboxes

In [4]:
target_gridboxes = {
          # lon_from, lon_to, lat_from, lat_to
    'park': [34.802, 34.805, 31.962, 31.965],
    'museum': [34.78, 34.785, 31.966, 31.97],
    'supermarket': [34.818, 34.820, 31.958, 31.963]
}

get all the siri ride stops which stopped in the grid box in the given date/time range

In [5]:
origin_grid_box_siri_ride_stops = list(stride.iterate('/siri_ride_stops/list', {
    'siri_vehicle_location__recorded_at_time_from': datetime_from,
    'siri_vehicle_location__recorded_at_time_to': datetime_to,
    'siri_vehicle_location__lon__greater_or_equal': origin_gridbox_lon_from,
    'siri_vehicle_location__lon__lower_or_equal': origin_gridbox_lon_to,
    'siri_vehicle_location__lat__greater_or_equal': origin_gridbox_lat_from,
    'siri_vehicle_location__lat__lower_or_equal': origin_gridbox_lat_to,
}))
pd.DataFrame(origin_grid_box_siri_ride_stops)

,id,siri_stop_id,siri_ride_id,order,gtfs_stop_id,nearest_siri_vehicle_location_id,siri_stop__code,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,...,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters,gtfs_ride_stop__gtfs_stop_id,gtfs_ride_stop__gtfs_ride_id,gtfs_ride_stop__arrival_time,gtfs_ride_stop__departure_time,gtfs_ride_stop__stop_sequence,gtfs_ride_stop__pickup_type,gtfs_ride_stop__drop_off_type,gtfs_ride_stop__shape_dist_traveled
0,100022605,2661,3907480,15,1315329,205957559,33587,1061,2022-04-03-56637570,2022-04-03T07:10:00+00:00,...,4996,215,1315329,4133452,2022-04-03T07:24:56+00:00,2022-04-03T07:24:56+00:00,15,0,0,4826
1,100064620,5029,3908852,16,1315330,206042331,33588,1061,2022-04-03-56637571,2022-04-03T07:25:00+00:00,...,5189,95,1315330,4133407,2022-04-03T07:40:38+00:00,2022-04-03T07:40:38+00:00,16,0,0,5189
2,100096819,4799,3906221,25,1315334,206105904,33592,1277,2022-04-03-36877185,2022-04-03T07:00:00+00:00,...,22142,77,1315334,4122875,2022-04-03T07:46:09+00:00,2022-04-03T07:46:09+00:00,25,1,0,22142
3,100123325,4799,3908222,28,1315334,206160346,33592,1640,2022-04-03-27565616,2022-04-03T07:20:00+00:00,...,11547,73,1315334,4113543,2022-04-03T07:56:59+00:00,2022-04-03T07:56:59+00:00,28,0,0,11547
4,100159391,4799,3909115,28,1315334,206232454,33592,1640,2022-04-03-34230644,2022-04-03T07:30:00+00:00,...,11547,91,1315334,4113533,2022-04-03T08:06:59+00:00,2022-04-03T08:06:59+00:00,28,0,0,11547
5,100173677,4799,3910071,28,1315334,206260996,33592,1640,2022-04-03-27565617,2022-04-03T07:40:00+00:00,...,11547,31,1315334,4113544,2022-04-03T08:16:59+00:00,2022-04-03T08:16:59+00:00,28,0,0,11547
6,100176218,5029,3913009,22,1315330,206265827,33588,1612,2022-04-03-39551004,2022-04-03T08:10:00+00:00,...,7027,89,1315330,4113490,2022-04-03T08:31:10+00:00,2022-04-03T08:31:10+00:00,22,0,0,7027
7,100180771,4799,3908648,42,1315334,206274536,33592,1064,2022-04-03-56637976,2022-04-03T07:25:00+00:00,...,17633,99,1315334,4133490,2022-04-03T08:18:28+00:00,2022-04-03T08:18:28+00:00,42,0,0,17633
8,100204861,4799,3910032,42,1315334,206322855,33592,1064,2022-04-03-56637977,2022-04-03T07:40:00+00:00,...,17633,34,1315334,4133466,2022-04-03T08:33:28+00:00,2022-04-03T08:33:28+00:00,42,0,0,17633
9,100260230,4799,3913697,28,1315334,206433694,33592,1640,2022-04-03-27565618,2022-04-03T08:20:00+00:00,...,11547,82,1315334,4113545,2022-04-03T08:56:59+00:00,2022-04-03T08:56:59+00:00,28,0,0,11547


get all unique siri rides, and get all their ride stops, split per ride so we get all siri ride stops per ride, including ride stops outside the origin grid box

finally we focus on a single siri_ride on which we will continue the processing

In [6]:
siri_rides = {
    siri_ride_stop['siri_ride_id']: {
        'siri_ride_id': siri_ride_stop['siri_ride_id'],
        'siri_ride_stops': []
    }
    for siri_ride_stop in origin_grid_box_siri_ride_stops
}
for siri_ride_stop in stride.iterate('/siri_ride_stops/list', {
    'siri_vehicle_location__recorded_at_time_from': datetime_from,
    'siri_vehicle_location__recorded_at_time_to': datetime_to,
    'siri_ride_ids': ','.join(map(str, siri_rides.keys()))
}):
    siri_ride = siri_rides[siri_ride_stop['siri_ride_id']]
    siri_ride['siri_ride_stops'].append(siri_ride_stop)
for siri_ride in siri_rides.values():
    siri_ride['siri_ride_stops'] = sorted(siri_ride['siri_ride_stops'], key=lambda siri_ride: siri_ride['order'])
focus_siri_ride = siri_ride
pd.DataFrame([{
    k: v for k, v in srs.items() if k.startswith('nearest_siri') or k.startswith('siri_ride') or k in ['order']
} for srs in focus_siri_ride['siri_ride_stops']])

,siri_ride_id,order,nearest_siri_vehicle_location_id,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,siri_ride__vehicle_ref,siri_ride__updated_first_last_vehicle_locations,siri_ride__first_vehicle_location_id,siri_ride__last_vehicle_location_id,...,siri_ride__gtfs_ride_id,nearest_siri_vehicle_location__siri_snapshot_id,nearest_siri_vehicle_location__siri_ride_stop_id,nearest_siri_vehicle_location__recorded_at_time,nearest_siri_vehicle_location__lon,nearest_siri_vehicle_location__lat,nearest_siri_vehicle_location__bearing,nearest_siri_vehicle_location__velocity,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters
0,3924010,1,206766371,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68006,100418161,2022-04-03T10:05:00+00:00,34.775410,31.988743,296,0,0,85
1,3924010,2,206787127,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68010,100439090,2022-04-03T10:13:29+00:00,34.772289,31.981842,128,0,1022,10
2,3924010,3,206792263,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68011,100441586,2022-04-03T10:14:41+00:00,34.778297,31.979790,104,25,1594,268
3,3924010,4,206797433,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68012,100444193,2022-04-03T10:15:40+00:00,34.780090,31.979334,104,0,1808,5
4,3924010,5,206807905,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68014,100449464,2022-04-03T10:17:35+00:00,34.786098,31.981855,18,32,2611,306
5,3924010,6,206813093,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68015,100452072,2022-04-03T10:18:37+00:00,34.786938,31.984159,28,18,2925,136
6,3924010,7,206818259,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68016,100454698,2022-04-03T10:19:40+00:00,34.787323,31.985485,6,0,3087,7
7,3924010,8,206823522,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68017,100457371,2022-04-03T10:20:38+00:00,34.788010,31.987619,92,18,3394,16
8,3924010,9,206828773,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68018,100460114,2022-04-03T10:21:36+00:00,34.789928,31.984989,170,43,3673,112
9,3924010,11,206833958,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68019,100463117,2022-04-03T10:23:12+00:00,34.789257,31.980587,174,32,4238,177


get only the ride stops on this route which occured at or after the origin grid box (as we can't travel back in time)

In [7]:
min_order = None
min_recorded_at_time = None
for srs in origin_grid_box_siri_ride_stops:
    if srs['siri_ride_id'] == focus_siri_ride['siri_ride_id']:
        if min_order is None or min_order > srs['order']:
            min_order = srs['order']
            min_recorded_at_time = datetime.datetime.strptime(srs['nearest_siri_vehicle_location__recorded_at_time'], '%Y-%m-%dT%H:%M:%S%z')
print('min_order:', min_order)
print('min_recorded_at_time:', min_recorded_at_time)
relevant_ride_stops = []
for srs in focus_siri_ride['siri_ride_stops']:
    if srs['order'] >= min_order:
        relevant_ride_stops.append(srs)
pd.DataFrame([{k: v for k, v in srs.items() if k.startswith('nearest_siri') or k.startswith('siri_ride') or k in ['order']} for srs in relevant_ride_stops])

min_order: 28
min_recorded_at_time: 2022-04-03 10:54:06+00:00


,siri_ride_id,order,nearest_siri_vehicle_location_id,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,siri_ride__vehicle_ref,siri_ride__updated_first_last_vehicle_locations,siri_ride__first_vehicle_location_id,siri_ride__last_vehicle_location_id,...,siri_ride__gtfs_ride_id,nearest_siri_vehicle_location__siri_snapshot_id,nearest_siri_vehicle_location__siri_ride_stop_id,nearest_siri_vehicle_location__recorded_at_time,nearest_siri_vehicle_location__lon,nearest_siri_vehicle_location__lat,nearest_siri_vehicle_location__bearing,nearest_siri_vehicle_location__velocity,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters
0,3924010,28,207000498,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68050,100545130,2022-04-03T10:54:06+00:00,34.808620,31.961990,82,25,11547,40
1,3924010,29,207005912,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68051,100547944,2022-04-03T10:55:22+00:00,34.810879,31.962217,86,0,11750,29
2,3924010,30,207011524,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68052,100550861,2022-04-03T10:56:23+00:00,34.813343,31.962370,80,0,11937,20
3,3924010,31,207017181,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68053,100553448,2022-04-03T10:57:10+00:00,34.816315,31.962589,58,0,12224,15
4,3924010,32,207022821,1640,2022-04-03-39551439,2022-04-03T10:10:00+00:00,7762869,2022-04-03 12:04:02.112544+00:00,206745747,207095187,...,4113581,68054,100555989,2022-04-03T10:58:18+00:00,34.819378,31.962267,102,0,12541,4


check if within those ride stops we get a hit in one of the target grid boxes

In [8]:
for srs in relevant_ride_stops:
    srs_lon = srs['nearest_siri_vehicle_location__lon']
    srs_lat = srs['nearest_siri_vehicle_location__lat']
    for name, tgb in target_gridboxes.items():
        tgb_lon_from, tgb_lon_to, tgb_lat_from, tgb_lat_to = tgb
        if tgb_lon_from <= srs_lon <= tgb_lon_to and tgb_lat_from <= srs_lat <= tgb_lat_to:
            print('hit', name, srs_lon, srs_lat)
            print('duration from origin (minutes)', (
                datetime.datetime.strptime(srs['nearest_siri_vehicle_location__recorded_at_time'], '%Y-%m-%dT%H:%M:%S%z') 
                - min_recorded_at_time
            ).total_seconds() / 60)
            break

hit supermarket 34.819378 31.962267
duration from origin (minutes) 4.2


lucky, got a hit on the supermarket, just 4.2 minutes from origin

but now we want more hits from the other targets too, so we will map each ride stop to a grid box which we can then use to call this algo again to get hits from that grid box as origin

In [9]:
processed_gridboxes = set()
for srs in relevant_ride_stops:
    srs_lon = srs['nearest_siri_vehicle_location__lon']
    srs_lat = srs['nearest_siri_vehicle_location__lat']
    if not (origin_gridbox_lon_from <= srs_lon <= origin_gridbox_lon_to and origin_gridbox_lat_from <= srs_lat <= origin_gridbox_lat_to):
        # this ride stop is not within the origin grid box
        lon_from = math.floor(srs_lon*1000)/1000
        lat_from = math.floor(srs_lat*1000)/1000
                 # lon_from, lon_to,        lat_from, lat_to
        gridbox = lon_from, lon_from+0.001, lat_from, lat_from+0.001
        if gridbox not in processed_gridboxes:
            processed_gridboxes.add(gridbox)
            print('process gridbox', gridbox, srs['nearest_siri_vehicle_location__recorded_at_time'])

process gridbox (34.81, 34.811, 31.962, 31.963) 2022-04-03T10:55:22+00:00
process gridbox (34.813, 34.814, 31.962, 31.963) 2022-04-03T10:56:23+00:00
process gridbox (34.816, 34.817, 31.962, 31.963) 2022-04-03T10:57:10+00:00
process gridbox (34.819, 34.82, 31.962, 31.963) 2022-04-03T10:58:18+00:00
